In [2]:
import sys
import pandas as pd
import numpy as np

# INPUT FILE
scen_file = 'esg_시나리오_20200518.xlsx'
in_file = 'ESG지수 백데이터_20200520.xlsx'
out_file_1 = 'convert_by_isu_'+ in_file   # xlsx로만 저장되네...
out_file_2 = 'convert_by_val_'+ in_file   # xlsx로만 저장되네...

# 시나리오에서 리벨런싱 시작일날짜를 가져온다.
df = pd.read_excel(scen_file,dtype='unicode')
df = df['정기개편일']
df.drop_duplicates(inplace=True)
phase = df.values.tolist()

print ('[START] converting 작업을 시작합니다.')

# input file 을 읽는다
#df = pd.read_csv(in_file, sep=',',dtype='unicode', parse_dates=True)
df = pd.read_excel(in_file, sep=',',dtype='unicode')

#숫자로 봐야하는 컬럼의 dtype을 변경한다
cols=df.columns.drop(['DATE','CODE'])
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# 필요없는 컬럼 지우기
#df.drop([df.columns[1],df.columns[2],df.columns[4],df.columns[7],df.columns[10],df.columns[11]],axis=1,inplace=True)

#
df.set_index('DATE', inplace=True)
df.sort_index(axis = 0, level=0, inplace=True, ascending=True)

# START - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목
# 각 개편일별로 끊어서 list에 추가함
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # TABLE RESHAPE - 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['CODE'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    df2['BAS_PRC'].apply(pd.to_numeric, errors='ignore')
    df_list.append(df2)
# 엑셀파일저장
with pd.ExcelWriter(out_file_2) as writer:
    for item in df_list :
        item.to_excel(writer, sheet_name=item.index[0])
        
print ('** FILE CREATED : ' + out_file_2)
# END - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목

#완료
print ('[END] converting 작업을 완료하였습니다.')



[START] converting 작업을 시작합니다.
** FILE CREATED : convert_by_val_ESG지수 백데이터_20200520.xlsx
[END] converting 작업을 완료하였습니다.


In [3]:
rebal_prc = 100000000 # 1억부터 시작

simple_index = []
contrb_list = []
cnt = 0
for my_df in df_list :
    adj_prc = my_df['ADJ_PRC']
    comp_rt = my_df['COMP_RT']

    #구간의 첫데이터에서 조목별 비중을 가져오고, 직전 구간의 마지막 가격합을 곱한다.
    invst_init = my_df.iloc[:1]
    
    #비중검증 (합이 1인가)
    #sum=invst_init['COMP_RT'].sum(axis = 1)
    #print(sum)

    invst_amt = invst_init['COMP_RT'].mul(rebal_prc)
    #첫산출일은 당일의 수정주가(종가)로 나누어 투자주식수를 구하고,
    #다음 리벨런싱부터는 전일의 수정주가로 나누어 투자주식수를 구한다.
    if cnt == 0 :
        invst_shrs = invst_amt.div(invst_init['ADJ_PRC'])
    else :
        invst_shrs = invst_amt.div(invst_init['PREV_ADJ_PRC'])

    # 위에서 구한 주식수를 수정주가 df에 곱하여, 일별 평가금액을 구한다
    cur_amt = pd.DataFrame(adj_prc.values * invst_shrs.values, index=adj_prc.index, columns=adj_prc.columns)

    # 기여도 구하기
    # 기간내 마지막일의 종가 기준 평가금액과 시작일의 투자금액간의 종목별 수익률을 구한 후,
    # 종목별 투자비중을 곱한다.
    # - 종목별투자금액 : invst_amt
    # - 종가기준평가금액 : cur_amt.iloc[-1:]  => 마지막값가져오기!
    # - 종목별 투자비중 : invst_init['COMP_RT']
    contribution = pd.DataFrame(cur_amt.iloc[-1:].values / invst_amt.values - 1, index=invst_amt.index,columns=invst_amt.columns)
    contribution = contribution.mul(invst_init['COMP_RT'])

    # 컬럼에 있는 종목코드를 행으로 내린다...어렵네...
    contribution = contribution.reset_index().melt(id_vars=['DATE']).set_index(['DATE'])
    contribution.sort_values(by=['value'], inplace=True)
    contrb_list.append(contribution)
    # 종목의 평가금액을 합한다.
    total_amt = cur_amt.sum(axis = 1)

    # index 리스트에 해당 리밸런싱구간의 데이터를 추가
    simple_index.append(total_amt)

    # 다음리밸런싱일에 사용할 투자금액을 가져온다. 해당 구간의 마지막 row 값
    rebal_prc = total_amt.iloc[-1:].values[0] 
    cnt = cnt + 1

result = pd.concat(simple_index)

result.to_excel("simple_index.xlsx")

result2 = pd.concat(contrb_list)
result2.to_excel("contributioin.xlsx")

print("수정주가를 이용한 지수생성 완료!")
print("output : " + "simple_index.xlsx")


수정주가를 이용한 지수생성 완료!
output : simple_index.xlsx


In [41]:
# full number 보이게 하기
pd.options.display.float_format = "{:.2f}".format

# 비교시가총액 검증
df1 = df_list[12]
df1.columns
#BAS_PRC', 'ADJ_PRC', '대비', 'PREV_ADJ_PRC', 'CUR_PRC', 'COMP_RT', 'STYLE_RT', 'SHRS', 'CHG_SHRS'
MKTCAP = df1['CUR_PRC'].mul(df1['STYLE_RT']).mul(df1['SHRS'])
#MKTCAP['20180627':'20180628'].to_excel("mktcap_test.xlsx")
MKTCAPSUM = MKTCAP.sum(axis=1)

In [45]:
df1['ADJ_PRC']['20191017':'20191018'].to_excel("adjprc_20191018.xlsx")
#MKTCAP['20191017':'20191018'].to_excel("mktcap_20191018.xlsx")

In [46]:
# ['BAS_PRC', 'ADJ_PRC', 'DIFF', 'PREV_ADJ_PRC', 'CUR_PRC', 'COMP_RT', 'STYLE_RT', 'SHRS', 'CHG_SHRS']
idx = pd.IndexSlice
F = df1.loc[:,idx[['ADJ_PRC','CUR_PRC','SHRS','CHG_SHRS'], ['036420'] ]]
F


,ADJ_PRC,CUR_PRC,SHRS,CHG_SHRS
CODE,036420,036420,036420,036420
DATE,,,,
20191001,35650.00,35650,14406898,0
20191002,33250.00,33250,14406898,0
20191004,32250.00,32250,14406898,0
20191007,33050.00,33050,14406898,0
20191008,33200.00,33200,14406898,0
20191010,32450.00,32450,14406898,0
20191011,34000.00,34000,14406898,0
20191014,35550.00,35550,14406898,0


In [35]:
a = 33099640 - 29176998
b = -218827485000
c = 472678361000
d = 29176998 - 33099640

In [40]:
m = 29176998 * 0.15
m

4376549.7

In [121]:
#contrb_list[14]
rslttt = pd.concat(contrb_list)

In [75]:
# stack 사용. series로 리턴이 와서 자신없다...
test=contribution.stack()
test.sort_values(inplace=True)
test.to_excel("consss.xlsx")

In [92]:
#df.reset_index().melt(id_vars=['Letters']).set_index(['Letters', 'variable'])
test = contribution.reset_index().melt(id_vars=['DATE']).set_index(['DATE','CODE'])
test.sort_values(by=['value'])

value
DATE     CODE                
20200102 009540 -1.863822e-02
         096770 -1.815375e-02
         034730 -1.508136e-02
         002790 -1.441998e-02
         000240 -1.322038e-02
         267250 -1.213586e-02
         006840 -1.116705e-02
         060980 -1.112723e-02
         006260 -1.060752e-02
         078930 -9.863252e-03
         003550 -8.149404e-03
         057050 -7.638691e-03
         000640 -7.314301e-03
         004800 -6.452930e-03
         001040 -5.656326e-03
         192400 -5.303292e-03
         001800 -5.254567e-03
         008930 -5.130091e-03
         000070 -4.946312e-03
         096760 -3.855748e-03
         012630 -3.281863e-03
         027410 -2.976085e-03
         015860 -2.624740e-03
         036420 -2.076149e-03
         004990 -1.206045e-03
         030530 -1.165433e-03
         101060 -7.302792e-04
         024720 -3.761201e-04
         063080 -1.435406e-04
         084110 -8.604659e-05
         003300 -2.546564e-18
         002020  1.063817e-04
         006120  9.289606e-04
         000140  2.191501e-03
         001630  3.736462e-03
         180640  3.988353e-03
         003090  5.324808e-03
         017810  6.951320e-03

In [86]:
start = cur_amt.iloc[:1]
end = cur_amt.iloc[-1:]
end.to_excel("end.xlsx")
invst_amt.to_excel("start.xlsx")

PermissionError: [Errno 13] Permission denied: 'end.xlsx'

In [35]:
contribution = pd.DataFrame(end.values / invst_amt.values - 1, index=start.index,columns=start.columns)
contribution.to_excel("contribution.xlsx")
invst_init['COMP_RT'].to_excel("comp_rt.xlsx")

In [36]:
result = contribution.mul(invst_init['COMP_RT'])
result.to_excel("result.xlsx")

In [30]:
invst_init['COMP_RT']

CODE,000070,000140,000240,000640,001040,001630,001800,002020,002790,003090,...,060980,063080,078930,084110,096760,096770,101060,180640,192400,267250
DATE,,,,,,,,,,,,,,,,,,,,,
20200102,0.018883,0.017532,0.040505,0.037802,0.031046,0.013478,0.028343,0.002667,0.043208,0.014829,...,0.029694,0.001316,0.0351,0.006721,0.020235,0.049964,0.008073,0.004018,0.033748,0.041856


In [156]:
#phase 4
rebal_prc = total_amt.iloc[-1:].values[0] 
prev_prc = df_list[3]['PREV_ADJ_PRC']
adj_prc = df_list[3]['ADJ_PRC']
comp_rt = df_list[3]['COMP_RT']

#구간의 첫데이터 접근
invst_init = df_list[3].iloc[:1]
invst_amt = invst_init['COMP_RT'].mul(rebal_prc)
invst_shrs = invst_amt.div(invst_init['PREV_ADJ_PRC'])
cur_amt = pd.DataFrame(adj_prc.values * invst_shrs.values, index=adj_prc.index, columns=adj_prc.columns)
total_amt = cur_amt.sum(axis = 1)

total_amt.to_excel("ok4.xlsx")

참고. SHAPE이 다른 Dataframe 끼리 곱하는법

In [65]:
# 추천 1. cur_amt라는 dataframe으로 옮긴다.
cur_amt[:] = adj_prc.values * invst_shrs.values

# 다른방법 
cur_amt2 =pd.DataFrame(adj_prc.values * invst_shrs.values, columns=adj_prc.columns)

# 다른방법: 그냥 values끼리 곱하면, nparray로 리턴값을 준다
cur_nparray = adj_prc.values * invst_shrs.values

In [55]:
cur_amt.head(5)

CODE,000070,000140,000240,000640,001040,001940,002020,002030,002250,002790,...,077360,078930,084690,096760,096770,101060,102260,108790,126560,180640
DATE,,,,,,,,,,,,,,,,,,,,,
20161004,2956810.6,2535991.1,4008859.4,3482835.0,3798449.6,3588039.9,1063122.9,431893.7,326688.8,4324474.0,...,116279.1,4114064.2,642303.4,2220376.5,4534883.7,1904761.9,2851605.8,747508.3,1589147.3,537098.6
20161005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20161006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20161007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20161010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
